# tket benchmarking example


Warning: This notebook is now deprecated for versions from 0.2.0 onwards, and is not going to be updated in ongoing releases.

The aim of this example is to show how to run the IBM benchmarking circuits through tket. You will need both `pytket` and `pytket_qiskit` installed from pip before running this turoial. You will also need `pandas` to capture the data.

The benchmarking circuits originated from https://github.com/iic-jku/ibm_qx_mapping/tree/master/examples, but there is a copy in pytket in the "benchmarking" folder. The initial circuits are written in QASM, meaning that they must be converted to tket's internal representation via Qiskit. Using this script we will compile these circuits through tket and then print a table to terminal containing analysis of the circuits post-compilation.

First, begin by importing qiskit and pytket.

In [1]:
import qiskit
import pytket

We need access to Qiskit's `QuantumCircuit` and the `circuit_to_dag` converter, which will allow us to take the original QASM and retrieve a Qiskit DAGCircuit. There is then a pytket method `dagcircuit_to_tk` to convert this DAGCircuit to tket's internal representation. We also need the pytket method `coupling_to_arc` to convert IBM's architectures to the tket `Architecture` class for the use of routing.

In [2]:
from qiskit import QuantumCircuit
from pytket.extensions.qiskit import IBMQBackend

from pytket.extensions.qiskit import qiskit_to_tk
from pytket.passes import PlacementPass, RoutingPass
from pytket.predicates import CompilationUnit
from pytket.routing import GraphPlacement


We need all of the required pytket equipment to allow us to perform clean-up transform passes, to route and to analyse the circuits. Lastly, we require the `pandas` module to hold our data, and 
`time` to benchmark compilation time per circuit.

The architectures used to benchmark for our routing paper were the IBMqx5 and IBMTokyo architectures. The architectures qx5 and Tokyo respectively are shown below (although the diagrams do not show the directedness of the coupling maps). These diagrams are from https://www.research.ibm.com/ibm-q/technology/devices/#ibmqx5. We will now define the coupling map representing both architectures. These will later be converted to directed graphs to be used by tket.
![alt text](IBMqx5Arc.png "Title")
![alt text](IBMTokyoArc.png "Title2")

In [3]:
from pytket.routing import Architecture, route
from pytket.transform import Transform
from pytket import OpType
from pytket import circuit
from pytket.circuit.display import render_circuit_jupyter
from pytket.routing import Placement
import pandas
import time

In [4]:
ibm_devices = {
    "ibmqx5": {"edges": [(1, 0), (1, 2), (2, 3), (3, 4), 
    (3, 14), (5, 4), (6, 5), (6, 7), (6, 11), (7, 10), 
    (8, 7), (9, 8), (9, 10), (11, 10), (12, 5), (12, 11), 
    (12, 13), (13, 4), (13, 14), (15, 0), (15, 2), (15, 14)],
    "nodes": 16},
    "ibmq_20_tokyo": {"edges": [(0, 1), (0, 5), (1, 2), (1, 6), (1, 7), (2, 3), (2, 6),
    (2, 7), (3, 4), (3, 8), (3, 9), (4, 8), (4, 9), (5, 6), (5, 10), (5, 11), (6, 7),
    (6, 10), (6, 11), (7, 8), (7, 12), (7, 13), (8, 9), (8, 12), (8, 13), (9, 14), (10, 11),
    (10, 15), (11, 12), (11, 16), (11, 17), (12, 13), (12, 16), (12, 17), (13, 14), (13, 18),
    (13, 19), (14, 18), (14, 19), (15, 16), (16, 17), (17, 18), (18, 19)], "nodes": 20},
    "Rochester" : {"edges": [(0, 1), (0, 5), (1, 2), (2, 3), (3, 4), (4, 6), (5, 9),
    (6, 13), (7, 8), (16, 7), (16, 19), (8, 9),  (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (11, 17),
    (15, 18), (18, 27), (19, 20), (20, 21), (21, 22), (22, 23), (17, 23), (23, 24), (24, 25), (25, 26), (26, 27),
    (21, 28), (28, 32), (25, 29), (29, 36), (30, 31), (31, 32), (32, 33), (33, 34), (34, 35), (35, 36), (36, 37), 
    (37, 38), (30, 39), (39, 42), (34, 40), (38, 41), (40, 46), (41, 50), (42, 43), (43, 44), (44, 45), 
    (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (48, 52), (44, 51)], "nodes": 53}
}

We shall now choose the device and create a directed graph for tket's routing to use.

In [5]:
device_name = 'ibmq_20_tokyo' ###Note: can also be ran using the Tokyo machine architecture
                                       ###      or with a user-defined coupling map
coupling_map = ibm_devices[device_name]["edges"]
directed_arc = Architecture(coupling_map)

We now define a method which takes in (1) a QASM file and (2) a directed graph architecture; it returns analysis of the circuit after our transform passes and routing procedure have been completed. It will also print out to terminal the time taken for all the transformations and routing to finish. Changing which optimisations are run in this method will trade off quality of the ouputs for time taken. For example, removing all optimisation passes and just running the routing procedure will give the fastest run time but may leave some redundant gates in the final circuit.

In [6]:
def getStats(filename, directed_arc):
    qc = QuantumCircuit.from_qasm_file(filename)
    tkcirc = qiskit_to_tk(qc)
    #cu = CompilationUnit(tkcirc)
    start_time = time.process_time()
    PlacementPass(GraphPlacement(directed_arc)).apply(tkcirc)
    RoutingPass(directed_arc).apply(tkcirc)
    #while (~DirectednessPredicate({}).verify(tkcirc)):
      #  RebaseTket().apply(tkcirc)
    # print(cu.initial_map)
    # print(cu.final_map)
    
    time_elapsed = time.process_time() - start_time
    
    print("Compilation time for circuit " + str(filename) + ": " + str(time_elapsed) + "s")
    
    #render_circuit_jupyter(tkcirc)
    if tkcirc.n_gates==0:
        return [0,0,0,0,0]
    ###Returns: [number of vertices, circuit depth, nubmer of CX gates, number of parallel slices of CX gates]
    return [tkcirc.n_gates, tkcirc.depth(), tkcirc.n_gates_of_type(OpType.CX), 
            tkcirc.depth_by_type(OpType.CX), time_elapsed]
    ###Note: the raw number of vertices in the circuits and the raw depth 
    ###      need to have the i/o vertices removed for fair comparisons

Lastly, we generate the table of input QASM filenames from a csv file using `pandas` and run the circuits through our compiler. The results are printed to terminal by default.

In [7]:
test_table = pandas.read_csv("benchmarking/IBMQConfig.csv",index_col=0)
test_table = test_table.sort_values(by='Depth in')

stat_table = pandas.DataFrame({})
total_time = 0

for i, (index, row) in enumerate(test_table.iterrows()):
    filename = row['Filename']
    new_stats = getStats(filename, directed_arc)
    total_time += new_stats[4] ###
    print("Time spent compiling so far: " + str(total_time))
    new_table_row = pandas.DataFrame.from_dict({index : new_stats}, 
    orient='index', columns=['Size out', 'Depth out', 'CX count out', 'CX depth out','Runtime'])
    stat_table = stat_table.append(new_table_row)

#stat_table.to_csv("BenchmarkTket.csv") ###Note: uncomment this line to print table to csv.
with pandas.option_context('display.max_rows', None):
    print(stat_table)

Compilation time for circuit benchmarking/ibmq/xor5_254.qasm: 0.0343239999999998s
Time spent compiling so far: 0.0343239999999998
Compilation time for circuit benchmarking/ibmq/graycode6_47.qasm: 0.06614999999999949s
Time spent compiling so far: 0.10047399999999929
Compilation time for circuit benchmarking/ibmq/ex1_226.qasm: 0.03374000000000077s
Time spent compiling so far: 0.13421400000000006
Compilation time for circuit benchmarking/ibmq/4gt11_84.qasm: 0.007169999999999455s
Time spent compiling so far: 0.1413839999999995
Compilation time for circuit benchmarking/ibmq/4mod5-v0_20.qasm: 0.01126899999999953s
Time spent compiling so far: 0.15265299999999904
Compilation time for circuit benchmarking/ibmq/ex-1_166.qasm: 0.006577000000000055s
Time spent compiling so far: 0.1592299999999991
Compilation time for circuit benchmarking/ibmq/4mod5-v1_22.qasm: 0.011538999999999966s
Time spent compiling so far: 0.17076899999999906
Compilation time for circuit benchmarking/ibmq/mod5d1_63.qasm: 0.012

Compilation time for circuit benchmarking/ibmq/4gt4-v0_80.qasm: 0.07242199999999954s
Time spent compiling so far: 3.1162099999999917
Compilation time for circuit benchmarking/ibmq/mod10_176.qasm: 0.1484179999999995s
Time spent compiling so far: 3.264627999999991
Compilation time for circuit benchmarking/ibmq/0410184_169.qasm: 0.3294950000000014s
Time spent compiling so far: 3.5941229999999926
Compilation time for circuit benchmarking/ibmq/qft_16.qasm: 0.27593200000000095s
Time spent compiling so far: 3.8700549999999936
Compilation time for circuit benchmarking/ibmq/4gt12-v0_88.qasm: 0.08607800000000054s
Time spent compiling so far: 3.956132999999994
Compilation time for circuit benchmarking/ibmq/rd84_142.qasm: 0.1719810000000006s
Time spent compiling so far: 4.128113999999995
Compilation time for circuit benchmarking/ibmq/rd53_311.qasm: 0.18427300000000102s
Time spent compiling so far: 4.312386999999996
Compilation time for circuit benchmarking/ibmq/4_49_16.qasm: 0.15261000000000102s
T

Time spent compiling so far: 82.38050000000001
Compilation time for circuit benchmarking/ibmq/inc_237.qasm: 9.564092000000002s
Time spent compiling so far: 91.94459200000001
Compilation time for circuit benchmarking/ibmq/cm85a_209.qasm: 9.48139599999999s
Time spent compiling so far: 101.425988
Compilation time for circuit benchmarking/ibmq/rd84_253.qasm: 11.758989000000014s
Time spent compiling so far: 113.18497700000002
Compilation time for circuit benchmarking/ibmq/co14_215.qasm: 22.496667000000002s
Time spent compiling so far: 135.681644
Compilation time for circuit benchmarking/ibmq/root_255.qasm: 14.566260999999997s
Time spent compiling so far: 150.247905
Compilation time for circuit benchmarking/ibmq/mlp4_245.qasm: 17.986597999999987s
Time spent compiling so far: 168.234503
Compilation time for circuit benchmarking/ibmq/urf2_277.qasm: 12.066460000000006s
Time spent compiling so far: 180.300963
Compilation time for circuit benchmarking/ibmq/sym9_148.qasm: 19.858160999999996s
Time 

KeyboardInterrupt: 